<a href="https://www.kaggle.com/code/vanpatangan/predict-future-sales?scriptVersionId=219783415" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# import libraries

In [ ]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns

from datetime import datetime

import warnings
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)
pd.options.display.float_format = '{:.2f}'.format

# load data

In [ ]:
sales_train = pd.read_csv('/kaggle/input/competitive-data-science-predict-future-sales/sales_train.csv')
items = pd.read_csv('/kaggle/input/competitive-data-science-predict-future-sales/items.csv')
item_categories = pd.read_csv('/kaggle/input/competitive-data-science-predict-future-sales/item_categories.csv')
shops = pd.read_csv('/kaggle/input/competitive-data-science-predict-future-sales/shops.csv')
test_df = pd.read_csv('/kaggle/input/competitive-data-science-predict-future-sales/test.csv')

# data summary

In [ ]:
def check(df):
    """
    Generates a concise summary of DataFrame columns.
    """
    # Compute values that are constant across columns
    total_rows = len(df)
    duplicates = df.duplicated().sum()

    # Use vectorized operations 
    dtypes = df.dtypes
    instances = df.count()
    unique = df.nunique()
    sum_null = df.isnull().sum()

    # Create the summary 
    df_check = pd.DataFrame({
        'column': df.columns,
        'dtype': dtypes,
        'instances': instances,
        'unique': unique,
        'sum_null': sum_null,
        'duplicates': duplicates  
    })

    return df_check

print("Sales Train")
display(check(sales_train))
display(sales_train.head())

print("Items")
display(check(items))
display(items.head())

print("Item Categories")
display(check(item_categories))
display(item_categories.head())

print("Shops")
display(check(shops))
display(shops.head())

print("Test")
display(check(test_df))
display(test_df.head())

# merge

In [ ]:
# Merge sales_train with items, item_categories, and shops
train = sales_train.merge(items, on='item_id', how='left') \
                   .merge(item_categories, on='item_category_id', how='left') \
                   .merge(shops, on='shop_id', how='left')

# Get train columns excluding the target variable
train_columns = [col for col in train.columns if col != "item_cnt_day"]

# Find missing columns in test_df
missing_cols = set(train_columns) - set(test_df.columns)

# Add missing columns to test_df with None (which becomes NaN in pandas)
for col in missing_cols:
    test_df[col] = None

# Make sure column order matches train (excluding target variable)
test = test_df[train_columns]

In [ ]:
# Convert date column to datetime 
train['date'] = pd.to_datetime(train['date'], dayfirst=True)
test['date'] = pd.to_datetime(test['date'], dayfirst=True)

# EDA visualization

In [ ]:
# Distribution plots for item_price and item_cnt_day
fig, axes = plt.subplots(1, 2, figsize=(14, 6))
sns.histplot(train['item_price'], bins=30, kde=True, ax=axes[0])
axes[0].set_title('Distribution of Item Price')
axes[0].set_xlabel('Item Price')

sns.histplot(train['item_cnt_day'], bins=30, kde=True, ax=axes[1])
axes[1].set_title('Distribution of Item Count per Day')
axes[1].set_xlabel('Item Count per Day')

plt.tight_layout()
plt.show()

In [ ]:
# Boxplots for outlier detection
fig, axes = plt.subplots(1, 2, figsize=(14, 6))
sns.boxplot(y=train['item_price'], ax=axes[0])
axes[0].set_title('Boxplot of Item Price')

sns.boxplot(y=train['item_cnt_day'], ax=axes[1])
axes[1].set_title('Boxplot of Item Count per Day')

plt.tight_layout()
plt.show()

In [ ]:
# Compute correlation matrix
correlation_matrix = train[['date_block_num', 'shop_id', 'item_id', 'item_price', 'item_cnt_day']].corr()

# Plot correlation heatmap
plt.figure(figsize=(8, 6))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt=".2f")
plt.title('Correlation Matrix')
plt.show()

In [ ]:
# Total sales per month
monthly_sales = train.groupby('date_block_num')['item_cnt_day'].sum().reset_index()
plt.figure(figsize=(10, 6))
sns.lineplot(data=monthly_sales, x='date_block_num', y='item_cnt_day')
plt.title('Total Sales Over Time (Monthly)')
plt.xlabel('Date Block Number')
plt.ylabel('Total Sales')
plt.show()

# Top 10 shops by total sales
top_shops = train.groupby('shop_name')['item_cnt_day'].sum().sort_values(ascending=False).head(10)
top_shops.plot(kind='bar', figsize=(10, 6), title='Top 10 Shops by Sales')
plt.ylabel('Total Sales')
plt.show()

# Top 10 categories by total sales
top_categories = train.groupby('item_category_name')['item_cnt_day'].sum().sort_values(ascending=False).head(10)
top_categories.plot(kind='bar', figsize=(10, 6), title='Top 10 Categories by Sales')
plt.ylabel('Total Sales')
plt.show()

# handle outliers

In [ ]:
# Calculate IQR for item_price
Q1_price = train['item_price'].quantile(0.25)
Q3_price = train['item_price'].quantile(0.75)
IQR_price = Q3_price - Q1_price
price_lower = Q1_price - 1.5 * IQR_price
price_upper = Q3_price + 1.5 * IQR_price

# Calculate IQR for item_cnt_day
Q1_cnt = train['item_cnt_day'].quantile(0.25)
Q3_cnt = train['item_cnt_day'].quantile(0.75)
IQR_cnt = Q3_cnt - Q1_cnt
cnt_lower = Q1_cnt - 1.5 * IQR_cnt
cnt_upper = Q3_cnt + 1.5 * IQR_cnt

# Cap outliers for item_price
train['item_price'] = train['item_price'].clip(lower=price_lower, upper=price_upper)

# Cap outliers for item_cnt_day
train['item_cnt_day'] = train['item_cnt_day'].clip(lower=cnt_lower, upper=cnt_upper)

# Plot for item_price
plt.figure(figsize=(10, 5))
plt.hist(train['item_price'], bins=50, color='blue', alpha=0.7)
plt.axvline(price_lower, color='red', linestyle='--', label='Lower Bound')
plt.axvline(price_upper, color='green', linestyle='--', label='Upper Bound')
plt.title('Distribution of item_price after capping')
plt.legend()
plt.show()

# Plot for item_cnt_day
plt.figure(figsize=(10, 5))
plt.hist(train['item_cnt_day'], bins=50, color='orange', alpha=0.7)
plt.axvline(cnt_lower, color='red', linestyle='--', label='Lower Bound')
plt.axvline(cnt_upper, color='green', linestyle='--', label='Upper Bound')
plt.title('Distribution of item_cnt_day after capping')
plt.legend()
plt.show()

# feature engineering

In [ ]:
def create_features(train, test):
    # Ensure consistent data types
    train['item_category_id'] = train['item_category_id'].fillna(-1).astype(int)
    test['item_category_id'] = test['item_category_id'].fillna(-1).astype(int)

    # Historical Aggregations
    train['shop_avg_sales'] = train.groupby('shop_id')['item_cnt_day'].transform('mean')
    train['category_avg_sales'] = train.groupby('item_category_id')['item_cnt_day'].transform('mean')

    # Merge aggregated features into test using latest available values
    shop_avg_sales = train.groupby('shop_id', as_index=False)['item_cnt_day'].mean().rename(columns={'item_cnt_day': 'shop_avg_sales'})
    category_avg_sales = train.groupby('item_category_id', as_index=False)['item_cnt_day'].mean().rename(columns={'item_cnt_day': 'category_avg_sales'})

    test = test.merge(shop_avg_sales, on='shop_id', how='left')
    test = test.merge(category_avg_sales, on='item_category_id', how='left')

    # Cumulative sum (test does not have sales history, so set to 0)
    train['cumulative_sales'] = train.groupby(['shop_id', 'item_id'])['item_cnt_day'].cumsum()
    test['cumulative_sales'] = 0  

    # Time-Based Features
    for df in [train, test]:
        df['month'] = df['date'].dt.month
        df['day_of_week'] = df['date'].dt.dayofweek
        df['is_weekend'] = (df['day_of_week'] >= 5).astype(int)

    # Lag Features (Only for train)
    train = train.sort_values(by=['shop_id', 'item_id', 'date'])
    train['lag_1'] = train.groupby(['shop_id', 'item_id'])['item_cnt_day'].shift(1)
    train['lag_12'] = train.groupby(['shop_id', 'item_id'])['item_cnt_day'].shift(12)

    # Forward-fill missing values for lags
    train[['lag_1', 'lag_12']] = train[['lag_1', 'lag_12']].fillna(method='ffill')

    # Use latest lag values from training data for test set
    latest_lags = train.groupby(['shop_id', 'item_id'], as_index=False)[['lag_1', 'lag_12']].last()
    test = test.merge(latest_lags, on=['shop_id', 'item_id'], how='left')

    # Final missing value handling
    train.fillna(0, inplace=True)
    test.fillna(0, inplace=True)

    # Ensure column order is consistent
    test = test[train.columns.drop('item_cnt_day', errors='ignore')]  # Remove 'item_cnt_day' if it exists

    return train, test

# Apply feature creation to both train and test dataframes
train, test = create_features(train, test)


In [ ]:
# To get the monthly we need to aggregate by date_block_num, item_id, and shop_id
item_cnt_month = train.groupby(['date_block_num', 'item_id', 'shop_id'])['item_cnt_day'].sum().reset_index()

# Rename column for clarity
item_cnt_month.rename(columns={'item_cnt_day': 'item_cnt_month'}, inplace=True)

# Merge item_cnt_month back to the main dataframe
train = train.merge(item_cnt_month, on=['date_block_num', 'item_id', 'shop_id'], how='left')

# Now item_cnt_month contains the total sales per item per shop per month

In [ ]:
# align test and merged_df except the target column item_cnt_day/ item_cnt_month

# Assuming you have 'train' and 'test' dataframes

# Get the column names as sets
train_columns = set(train.columns)
test_columns = set(test.columns)

# Find missing columns in test
missing_in_test = train_columns - test_columns
# Find missing columns in train
missing_in_train = test_columns - train_columns

# Print the results
print("Columns missing in test:")
print(missing_in_test)

print("\nColumns missing in train:")
print(missing_in_train)


In [ ]:
""""
# Copy Id for submission
test_id = test_df['ID'].copy()

# Identify columns present in train_df but not in test_df 
missing_columns = set(train_df.columns) - set(test_df.columns) - {'item_cnt_day'}

# Add missing columns to test_df with default values 0
for col in missing_columns:
    test_df[col] = 0  

# Ensure both dataframes have the same column order 
common_columns = [col for col in train_df.columns if col in test_df.columns and col != 'item_cnt_day']

# Reorder columns
train_df = train_df[common_columns + ['item_cnt_day']]
test_df = test_df[common_columns]

# Verify that the columns are aligned 
assert set(train_df.columns) == set(test_df.columns).union({'item_cnt_day'}), "Columns do not match"

In [ ]:
from sklearn.preprocessing import LabelEncoder

# Label encode object dtype columns
le = LabelEncoder()
object_cols = train.select_dtypes(include='object').columns
for col in object_cols:
    train[col] = le.fit_transform(train[col])
    test[col] = le.fit_transform(test[col])

In [ ]:
train.info()

# modeling

In [ ]:
import lightgbm as lgb
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from scipy.stats import randint, uniform

# Define the feature set and target variable
X = train.drop(['item_cnt_day', 'item_cnt_month', 'date', 'item_name', 'item_category_name', 'shop_name'], axis=1)
y = train['item_cnt_month']

# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Define parameter distribution for random search
param_dist = {
    'num_leaves': randint(20, 3000),
    'learning_rate': uniform(0.01, 0.3),
    'feature_fraction': uniform(0.5, 0.5),
    'bagging_fraction': uniform(0.5, 0.5),
    'bagging_freq': randint(1, 7),
    'min_child_samples': randint(10, 100),
    'max_depth': randint(3, 12),
    'n_estimators': randint(100, 1000)
}

# Initialize LightGBM regressor
lgb_estimator = lgb.LGBMRegressor(objective="regression", metric="rmse", 
                                  random_state=42)

# Perform Random Search with cross-validation
random_search = RandomizedSearchCV(
    estimator=lgb_estimator,
    param_distributions=param_dist,
    n_iter=100,  # Number of parameter settings that are sampled
    cv=3,
    scoring='neg_root_mean_squared_error',
    verbose=-1,
    n_jobs=-1,
    random_state=42
)

# Fit the Random Search
random_search.fit(X_train, y_train)

# Best parameters from Random Search
best_params = random_search.best_params_
print("Best parameters found by Random Search:", best_params)

# Train the final model with the best parameters
final_model = lgb.LGBMRegressor(**best_params, objective="regression", 
                                metric="rmse", random_state=42)
final_model.fit(X_train, y_train)

# Predict on the validation set
y_pred = final_model.predict(X_val)

# Evaluate the final model
mse_final = mean_squared_error(y_val, y_pred, multioutput='raw_values')
rmse_final = np.sqrt(mse_final)
r2_final = r2_score(y_val, y_pred, multioutput='raw_values')

print("Final Model RMSE:", rmse_final)
print("Final Model R-squared:", r2_final)

# Feature importance
feature_importance = final_model.feature_importances_
feature_names = X.columns
for importance, name in sorted(zip(feature_importance, feature_names), 
                               reverse=True):
    print(f"{name}: {importance}")


# predict on test set

In [ ]:
# Drop date
X_test = test_df.drop(['date'], axis=1)

# Check the test set has the same columns as the training set
missing_cols = set(X_train.columns) - set(X_test.columns)
for col in missing_cols:
    X_test[col] = 0
X_test = X_test[X_train.columns]

# Predict on the test dataset
y_test_pred = final_model.predict(X_test)

# submission

In [ ]:
# Save the predictions
predictions_df = pd.DataFrame({'id': test_id, 'item_cnt_month': y_test_pred})

# Save predictions to a CSV file
predictions_df.to_csv('submission.csv', index=False)

# Display the first few rows of the predictions
predictions_df.head()